In [11]:
import torch
from accelerate import Accelerator
import pandas as pd

# hardware optimization
accelerator = Accelerator(mixed_precision='fp16')

# get code, instrument, and pretrained spectrum model from the hub
github = "pmelchior/spender"
sdss, model = torch.hub.load(github, 'sdss_II',  map_location=accelerator.device)
model = model.to('cpu')
# get some SDSS spectra from the ids, store locally in data_path
from spender.data.sdss import SDSS
data_path = "./DATA"

key_df = pd.read_csv('/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/code/monthly/feb2024/wk4/key.csv')

key_df = key_df.sample(frac=1)
plates, fibers, MJDs = [key_df[i].to_numpy()[0:5000] for i in ['Plate', 'Fiber', 'MJD']]
ids = [(plates[i], MJDs[i], fibers[i]) for i in range(len(plates))]
spec, w, z, norm, zerr = SDSS.make_batch(data_path, ids)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Using cache found in /Users/yaroslav/.cache/torch/hub/pmelchior_spender_main


downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//2581/spec-2581-54085-0496.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//2493/spec-2493-54115-0275.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//0700/spec-0700-52199-0237.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//1722/spec-1722-53852-0087.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//2604/spec-2604-54484-0220.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//0996/spec-0996-52641-0002.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//2673/spec-2673-54096-0194.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//1055/spec-1055-52761-0122.fits
downloading https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/lite//0872/spec-0872-52339-0086.fits
downloadin

KeyboardInterrupt: 

In [28]:
plates_ = []
MJDs_ = []
fibers_ = []

root_dir = '/Users/yaroslav/Documents/2. work/Research/GitHub/AGN-UMAP/code/monthly/feb2024/wk4/DATA'

import os
for dir in os.listdir(root_dir):
    for file in os.listdir(os.path.join(root_dir, dir)):
        file_list = file.split('-')
        plates_.append(file_list[1])
        MJDs_.append(file_list[2])
        fibers_.append(file_list[3].split('.')[0])
ids_ = [(int(plates_[i]), int(MJDs_[i]), int(fibers_[i])) for i in range(len(plates_))]
spec, w, z, norm, zerr = SDSS.make_batch(data_path, ids_)

In [32]:
# run spender end-to-end
#with torch.no_grad():
#  spec_reco = model(spec, instrument=sdss, z=z)

# only encode into latents
with torch.no_grad():
  s = model.encode(spec)
  p = model.predict(spec)

AttributeError: 'SpectrumAutoencoder' object has no attribute 'predict'

In [31]:
import numpy as np

#s = s.numpy()

array([], shape=(539, 0), dtype=float32)

array([nan, nan, nan, nan, nan, nan], dtype=float32)

In [21]:
import umap 
import numpy as np
import matplotlib.pyplot as plt

reducer = umap.UMAP(n_components=2)
reduced_data = reducer.fit_transform(s)

# Plot the Reduced Data 

umap_x = reduced_data[:, 0]
umap_y = reduced_data[:, 1]

plt.scatter(umap_x, umap_y, s=5)
plt.title("UMAP Visualization of Encoded Data")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.show()

ValueError: Input contains NaN.